In [24]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_SECRET_KEY']

In [2]:
from langchain.agents import tool

In [10]:
from pydantic.v1 import Field, BaseModel

class GetNameInput(BaseModel):
    query: str = Field(..., title="Name", description="Query entered by the user which contains the name of the name of the movie or the TV show") 

In [11]:
print(issubclass(GetNameInput, BaseModel))

True


In [12]:
@tool(args_schema=GetNameInput)
def get_name(query : str) -> str:
    """ Get the name of the movie or the TV show from the query"""
    return "The Matrix"

In [13]:
get_name.args

{'query': {'title': 'Name',
  'description': 'Query entered by the user which contains the name of the name of the movie or the TV show',
  'type': 'string'}}

In [14]:
class GetIdInput(BaseModel):
    name: str = Field(..., title="Name", description="Name of the movie or the TV show")

In [15]:
import requests
import json
@tool(args_schema=GetIdInput)
def get_id(name : str) -> str:
    """ Get the ID of the movie or the TV show from the name"""
    
    url = "https://imdb8.p.rapidapi.com/title/get-details"

    querystring = {"tconst":"tt0944947"}

    headers = {
        "X-RapidAPI-Key": "9bf945d759mshde4fe1a4916fc96p176ed9jsnd4cd2f06b55e",
        "X-RapidAPI-Host": "imdb8.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    # Extract the value associated with the key "id"
    id_value = data["id"]
    # Extract the actual ID without the prefix
    actual_id = id_value.split('/')[2]
    return actual_id

In [16]:
get_id.description

'get_id(name: str) -> str - Get the ID of the movie or the TV show from the name'

In [17]:
get_id.args

{'name': {'title': 'Name',
  'description': 'Name of the movie or the TV show',
  'type': 'string'}}

In [18]:
from langchain.tools.render import format_tool_to_openai_function

In [19]:
format_tool_to_openai_function(get_id)

/Users/jayantduneja/anaconda3/envs/imdb-chatbot/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'get_id',
 'description': 'get_id(name: str) -> str - Get the ID of the movie or the TV show from the name',
 'parameters': {'type': 'object',
  'properties': {'name': {'description': 'Name of the movie or the TV show',
    'type': 'string'}},
  'required': ['name']}}

In [20]:
get_id("The Matrix")

'tt0944947'

In [28]:
from langchain.chat_models import ChatOpenAI
functions = [
    format_tool_to_openai_function(f) for f in [get_name, get_id]
]
model = ChatOpenAI(temperature=0).bind(functions= functions)

In [29]:
model.invoke("What is the ID of the movie The Matrix?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "name": "The Matrix"\n}', 'name': 'get_id'}})

In [30]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful agent to get the ID of the movie or the TV show from the name"),
    ("user", "{input}"),
])
chain = prompt | model

In [31]:
chain.invoke({"input": "What is the ID of the movie The Matrix?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "name": "The Matrix"\n}', 'name': 'get_id'}})

In [33]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [34]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [35]:
result = chain.invoke({"input": "What is the ID of the movie The Matrix?"})

In [36]:
result.tool

'get_id'

In [37]:
result.tool_input

{'name': 'The Matrix'}

In [38]:
result = chain.invoke({"input": "Hi"})

In [39]:
result.return_values

{'output': 'Hello! How can I assist you today?'}

In [40]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "get_id" : get_id,
        }
        return tools[result.tool].run(result.tool_input)

In [41]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [42]:
result = chain.invoke({"input": "What is the ID of the movie The Matrix?"})

In [43]:
result

'tt0944947'

In [2]:
import streamlit as st

In [3]:
with st.chat_message("assistant"):
    st.write("Hi. I am a helpful agent which can answer any questions about movies or TV shows. What do you want to know?")

2024-02-05 10:30:23.897 
  command:

    streamlit run /Users/jayantduneja/anaconda3/envs/imdb-chatbot/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
